In [5]:
import pandas as pd
import distance
import os

In [11]:
def extract_adj_list(aa):
    
    df = pd.DataFrame.from_csv('../copies_free_db/' + aa + '/stems.txt', sep='\t')
    df['Sequence'] = df.Stem_1 + df.Stem_2 + df.Stem_3 + df.Stem_4
    df.drop(['Stem_1','Stem_2','Stem_3','Stem_4','Variable_stem'], axis=1, inplace=True)
    aa_list = list(set(df.Sequence))
    aa_adj_list = []

    for ind1 in range(len(aa_list)):
        temp = []
        for ind2 in range(len(aa_list)):
            if ind1 != ind2 and distance.hamming(aa_list[ind1], aa_list[ind2]) == 1:
                temp.append(ind2)
        aa_adj_list.append(temp)
    
    aa_freq = []

    for ind1 in range(len(aa_list)):
        counter = 0
        for ind2 in range(len(df)):
            if df.Sequence[ind2] == aa_list[ind1]:
                counter +=1
        aa_freq.append(counter)
    
    result = pd.DataFrame(columns = ['Adjacency','Frequency'], index = aa_list)
    result['Adjacency'] = aa_adj_list
    result['Frequency'] = aa_freq
    
    return result

In [ ]:
for aa in os.listdir('../copies_free_db/'):
    if '.' not in aa:
        print aa
        db = extract_adj_list(aa)
        db.to_csv('../data/'+aa+'.txt',sep='\t')

ile
gly
